## Xenium data analysis with spatial trajectories inference

We will go through the tutorial on how to use stLearn to perform clustering (with or without imputation) and spatial trajectories inference with Xenium data from 10X Genomics. The main dataset we are working on is the breast cancer tumor microenvironment. You can download it here: https://www.10xgenomics.com/products/xenium-in-situ/preview-dataset-human-breast 

### 1. Loading library

In [ ]:
import scanpy as sc
import stlearn as st

st.settings.set_figure_params(dpi=120)

# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
library_id = "Xenium_FFPE_Human_Breast_Cancer_Rep1"

In [ ]:
st.datasets.xenium_sge(library_id=library_id, include_hires_tiff=True)

### 2. Reading Xenium data

You can download all the data in the 10X Genomics resource webpage. For H&E image, because it was generated separately with the main xenium data, then you need to perform the registration by your self. Here we provide an example of a H&E image which is performed a manual registration by Dr. Soo Hee Lee. You can download it here: [Link](https://www.dropbox.com/s/th6tqqgbv27o3fk/CS1384_post-CS0_H%26E_S1A_RGB-shlee-crop.png?dl=1)

In [ ]:
data_dir = st.settings.datasetdir / "Xenium_FFPE_Human_Breast_Cancer_Rep1"

adata = st.ReadXenium(feature_cell_matrix_file=data_dir / "cell_feature_matrix.h5",
                      cell_summary_file=data_dir / "cells.csv.gz",
                      library_id=library_id,
                      image_path=data_dir / "he_image.ome.tif",
                      scale=1,
                      spot_diameter_fullres=15,
                      alignment_matrix_file=data_dir / "he_imagealignment.csv",
                      experiment_xenium_file=data_dir / "experiment.xenium",
                      )

In [ ]:
import numpy as np
import pandas as pd
import scanpy
from pathlib import Path
from PIL import Image
import stlearn
from typing import Union


def new_ReadXenium(
        feature_cell_matrix_file: Union[str, Path],
        cell_summary_file: Union[str, Path],
        image_path: Union[Path, None] = None,
        library_id: Union[str, None] = None,
        scale: float = 1.0,
        quality: str = "hires",
        spot_diameter_fullres: float = 15,
        background_color: str = "white",
        alignment_matrix_file: Union[str, Path, None] = None,
        pixel_size_microns: float = 0.2125,
        coordinate_space: str = "xenium"  # "xenium" or "image"
) -> "AnnData":
    """\
    Read Xenium data with optional H&E image alignment

    Parameters
    ----------
    feature_cell_matrix_file
        Path to feature cell count matrix file. Only support h5ad file now.
    cell_summary_file
        Path to cell summary CSV file.
    image_path
        Path to image. Only need when loading full resolution image.
    library_id
        Identifier for the Xenium library. Can be modified when concatenating multiple
        adata objects.
    scale
        Set scale factor.
    quality
        Set quality that convert to stlearn to use. Store in
        anndata.obs['imagecol' & 'imagerow']
    spot_diameter_fullres
        Diameter of spot in full resolution
    background_color
        Color of the background. Only `black` or `white` is allowed.
    alignment_matrix_file
        Path to transformation matrix CSV file exported from Xenium Explorer.
        If provided, coordinates will be transformed according to coordinate_space.
    pixel_size_microns
        Pixel size in microns (default 0.2125 for Xenium data).
    coordinate_space
        Target coordinate space: "xenium" (default) or "image".
        - "xenium": Convert image coordinates to Xenium space
        - "image": Convert Xenium coordinates to image space

    Returns
    -------
    AnnData
    """

    metadata = pd.read_csv(cell_summary_file)
    adata = scanpy.read_10x_h5(feature_cell_matrix_file)

    # Get original spatial coordinates
    spatial = metadata[["x_centroid", "y_centroid"]].copy()

    # Apply alignment transformation if provided
    if alignment_matrix_file is not None:
        spatial = apply_alignment_transformation(
            spatial,
            alignment_matrix_file,
            pixel_size_microns,
            coordinate_space
        )

    spatial.columns = ["imagecol", "imagerow"]
    adata.obsm["spatial"] = spatial.values

    if scale is None:
        max_coor = np.max(adata.obsm["spatial"])
        scale = 2000 / max_coor

    if library_id is None:
        library_id = "Xenium_data"

    adata.obs["imagecol"] = spatial["imagecol"].values * scale
    adata.obs["imagerow"] = spatial["imagerow"].values * scale

    if image_path is not None:
        stlearn.add.image(
            adata,
            library_id=library_id,
            quality=quality,
            imgpath=image_path,
            scale=scale,
        )
    else:
        # Create image
        max_size = np.max([adata.obs["imagecol"].max(), adata.obs["imagerow"].max()])
        max_size = int(max_size + 0.1 * max_size)

        if background_color == "black":
            image = Image.new("RGBA", (max_size, max_size), (0, 0, 0, 0))
        else:
            image = Image.new("RGBA", (max_size, max_size), (255, 255, 255, 255))
        imgarr = np.array(image)

        # Create spatial dictionary
        adata.uns["spatial"] = {}
        adata.uns["spatial"][library_id] = {}
        adata.uns["spatial"][library_id]["images"] = {}
        adata.uns["spatial"][library_id]["images"][quality] = imgarr
        adata.uns["spatial"][library_id]["use_quality"] = quality
        adata.uns["spatial"][library_id]["scalefactors"] = {}
        adata.uns["spatial"][library_id]["scalefactors"][
            "tissue_" + quality + "_scalef"
            ] = scale
        adata.uns["spatial"][library_id]["scalefactors"][
            "spot_diameter_fullres"
        ] = spot_diameter_fullres

    return adata


def apply_alignment_transformation(
        coordinates: pd.DataFrame,
        alignment_matrix_file: Union[str, Path],
        pixel_size_microns: float = 0.2125,
        coordinate_space: str = "xenium"
) -> pd.DataFrame:
    """
    Apply transformation matrix to convert coordinates between spaces.

    Parameters
    ----------
    coordinates
        DataFrame with columns ['x_centroid', 'y_centroid'] in microns
    alignment_matrix_file
        Path to transformation matrix CSV file from Xenium Explorer
    pixel_size_microns
        Pixel size in microns
    coordinate_space
        Target coordinate space: "xenium" or "image"

    Returns
    -------
    pd.DataFrame
        Transformed coordinates
    """

    # Read transformation matrix
    transform_mat = pd.read_csv(alignment_matrix_file, header=None).values

    # Convert coordinates based on target space
    if coordinate_space == "image":
        # Convert Xenium coordinates to image space
        # First convert microns to pixels
        coords_pixels = coordinates.values / pixel_size_microns

        # Use inverse transformation matrix
        transform_mat_inv = np.linalg.inv(transform_mat)

        # Apply transformation
        coords_homogeneous = np.column_stack([coords_pixels, np.ones(len(coords_pixels))])
        transformed_coords = np.dot(coords_homogeneous, transform_mat_inv.T)

        # Extract x, y coordinates (ignore homogeneous coordinate)
        result_coords = transformed_coords[:, :2]

    elif coordinate_space == "xenium":
        # Convert image coordinates to Xenium space
        # Assuming input coordinates are in pixels, convert to homogeneous coordinates
        coords_homogeneous = np.column_stack([coordinates.values, np.ones(len(coordinates))])

        # Apply transformation
        transformed_coords = np.dot(coords_homogeneous, transform_mat.T)

        # Extract x, y coordinates and convert to microns
        result_coords = transformed_coords[:, :2] * pixel_size_microns

    else:
        raise ValueError("coordinate_space must be 'xenium' or 'image'")

    # Return as DataFrame with original column names
    return pd.DataFrame(result_coords, columns=coordinates.columns)


def load_transformation_matrix(alignment_matrix_file: Union[str, Path]) -> np.ndarray:
    """
    Load and return transformation matrix from CSV file.

    Parameters
    ----------
    alignment_matrix_file
        Path to transformation matrix CSV file from Xenium Explorer

    Returns
    -------
    np.ndarray
        3x3 transformation matrix
    """
    return pd.read_csv(alignment_matrix_file, header=None).values


def convert_coordinates(
        coordinates: np.ndarray,
        transformation_matrix: np.ndarray,
        pixel_size_microns: float = 0.2125,
        from_space: str = "xenium",
        to_space: str = "image"
) -> np.ndarray:
    """
    Convert coordinates between Xenium and image spaces.

    Parameters
    ----------
    coordinates
        Array of coordinates (N x 2) in microns (if from Xenium) or pixels (if from image)
    transformation_matrix
        3x3 transformation matrix from Xenium Explorer
    pixel_size_microns
        Pixel size in microns
    from_space
        Source coordinate space: "xenium" or "image"
    to_space
        Target coordinate space: "xenium" or "image"

    Returns
    -------
    np.ndarray
        Transformed coordinates
    """

    if from_space == "xenium" and to_space == "image":
        # Convert microns to pixels
        coords_pixels = coordinates / pixel_size_microns

        # Use inverse transformation matrix
        transform_mat_inv = np.linalg.inv(transformation_matrix)

        # Apply transformation
        coords_homogeneous = np.column_stack([coords_pixels, np.ones(len(coords_pixels))])
        transformed_coords = np.dot(coords_homogeneous, transform_mat_inv.T)

        return transformed_coords[:, :2]

    elif from_space == "image" and to_space == "xenium":
        # Apply transformation
        coords_homogeneous = np.column_stack([coordinates, np.ones(len(coordinates))])
        transformed_coords = np.dot(coords_homogeneous, transformation_matrix.T)

        # Convert pixels to microns
        return transformed_coords[:, :2] * pixel_size_microns

    else:
        raise ValueError("Invalid coordinate space combination")


# Example usage:
"""
# Load Xenium data with H&E alignment
adata = ReadXenium(
    feature_cell_matrix_file="cell_feature_matrix.h5",
    cell_summary_file="cells.csv",
    image_path="he_image.jpg",
    alignment_matrix_file="imagealignment.csv",
    coordinate_space="image"  # Convert to image coordinate space
)

# Or manually convert coordinates
transform_matrix = load_transformation_matrix("imagealignment.csv")
xenium_coords = np.array([[5372.5, 12779.5]])  # Example coordinates in microns
image_coords = convert_coordinates(
    xenium_coords,
    transform_matrix,
    from_space="xenium",
    to_space="image"
)
"""

In [ ]:
adata

### 3. Preprocessing

Here, we use scanpy to perform several steps of preprocessing

In [ ]:
# Filter genes and cells with at least 10 counts
sc.pp.filter_genes(adata, min_counts=10)
sc.pp.filter_cells(adata, min_counts=10)

In [ ]:
# Store the raw data for using PSTS
adata.raw = adata

In [ ]:
# Normalization data
sc.pp.normalize_total(adata)

After normalization, we have two options. One is using log transformation or square root transformation (recommend by the 10X team)

In [ ]:
# Squareroot normalize transcript counts. We need to deal with sparse matrix of .X

adata.X = np.sqrt(adata.X.toarray()) + np.sqrt(adata.X.toarray() + 1)
# If the matrix is small, we don't need to convert to numpy array. I believe, Xenium data is always large 
# adata.X = csr_array(np.sqrt(adata.X) + np.sqrt(adata.X + 1))

# If you want to use log transformation, please use:
# st.pp.log1p(adata)

In [ ]:
# Run PCA
st.em.run_pca(adata, n_comps=50, random_state=0)

##### Apply stSME (optional)

stSME is optional as an imputation function. Due to the scope of this tutorial, we will not use it here, but we recommend you run if you have an H&E image in your dataset

In [ ]:
# apply stSME to normalise log transformed data
# st.spatial.SME.SME_normalize(adata, use_data="raw")
# adata.X = adata.obsm['raw_SME_normalized']
# st.pp.scale(adata)
# st.em.run_pca(adata,n_comps=50)

In [ ]:
# Compute neighborhood graph of cells using the PCA representation
st.pp.neighbors(adata, n_neighbors=25, use_rep='X_pca', random_state=0)

### 3. Clustering

Here we use `louvain` clustering. You also can use `leiden` clustering method with `scanpy.tl.leiden`.

In [ ]:
st.tl.clustering.louvain(adata, random_state=0)

We can plot the clustering result: 

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 10)
st.pl.cluster_plot(adata, use_label="louvain", image_alpha=1, size=1, cell_alpha=0.5, show_image=True, show_axis=True)

You can crop a region of interest with `zoom_coord` parameter.

In [ ]:
plt.rcParams["figure.figsize"] = (5, 5)
st.pl.cluster_plot(adata, use_label="louvain",
                   image_alpha=0.2, size=10,
                   cell_alpha=0.9,
                   zoom_coord=(17000.0, 18500.0, 7500.0, 8900.0),
                   show_color_bar=True)

### 4. PSeudo-Time-Space (PSTS) for spatial trajectory analysis

We provide PSTS for spatial trajectory analysis.

The first step is to select the root index. We allow the user to automatically select the index of the cell but still need to input which cluster should be the root cluster in your biological question.

In this tutorial, we focus on the cancer progression from Ductal carcinoma in situ (DCIS) to Invasive ductal carcinoma (IDC). That is why we choose the root cluster `6` as the DCIS cluster. 

In [ ]:
adata.uns["iroot"] = st.spatial.trajectory.set_root(
    adata,
    use_label="louvain",
    cluster="4",
    use_raw=True
)
adata.uns["iroot"]

For the `pseudotime` function, we are based on the diffusion pseudotime algorithm to initialize the pseudotime for trajectory at the gene expression level. `pseudotimespace_global` is the function for incorporating the spatial information to construct the (meta) trajectory in the spatial context.

In [ ]:
st.spatial.trajectory.pseudotime(adata, eps=50, use_rep="X_pca", use_label="louvain")

In this dataset, the IDC cluster we are looking for is the one on the edge of ducts. This part is the first layer of invasive cancer cells starting from DCIS. That's how we choose cluster `10` for the IDC.

In [ ]:
st.spatial.trajectory.pseudotimespace_global(adata, use_label="louvain", list_clusters=["4", "9"])

After construct the spatial trajectories from DCIS to IDC cluster. We can observe multiple meta-trajectories with sub-clusters which generated by spatial feature. With `tree_plot`, you can show all of the detail sub-clusters ids.

In [ ]:
st.pl.trajectory.tree_plot(adata, figsize=(10, 20), ncols=5)

Now, we can crop and discover the transition genes of sub-clusters meta-trajectory

In [ ]:
st.pl.cluster_plot(
    adata,
    use_label="louvain",
    show_trajectories=True,
    list_clusters=["4", "9"],
    show_subcluster=True,
    zoom_coord=(17000.0, 18500.0, 7500.0, 8900.0),
    show_node=False
)

Finding the transition markers of `clade_3019`

In [ ]:
st.spatial.trajectory.detect_transition_markers_clades(adata, clade=3019, use_raw_count=False, cutoff_spearman=0.1)

Plot the transition markers, red color is down-regulated gene, blue color is up-regulated gene

In [ ]:
plt.rcParams["figure.figsize"] = (5, 5)
st.pl.trajectory.transition_markers_plot(adata, top_genes=20, trajectory="clade_3019")

We can observe some keratinocytes family genes here as the progression markers.

### Acknowledgement

We would like to thank Soo Hee Lee and 10X Genomics team for their able support, contribution and feedback to this tutorial